In [ ]:
# import required libraries
import os
import numpy as np
from tqdm import tqdm
from glob import glob
from numpy import zeros
from numpy.random import randint
import torch
import os
import cv2
from statistics import mean
from torch.nn.functional import threshold, normalize

# Data Viz
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import cv2

Import segment_anything and its dependency

In [ ]:
! pip install torch torchvision &> /dev/null
! pip install opencv-python pycocotools matplotlib onnxruntime onnx &> /dev/null

# Download the ViT versions of the mobile SAM model weights from github repository.
!git clone https://github.com/ChaoningZhang/MobileSAM.git
%cd MobileSAM
!pip install -e .

Improt Training data



In [ ]:
# === CONFIGURATION ===
# Set the path to your training images and labels
image_path = "/path/to/train/images"   # <-- Replace with actual image folder path
label_path = "/path/to/train/labels"   # <-- Replace with actual label folder path

# === Load Image Paths ===
# Count total number of image files (e.g., .jpg format)
all_image_paths = sorted(glob(os.path.join(image_path, "*.jpg")))  # Use .png if needed
total_images = len(all_image_paths)
print(f"Total Number of Images: {total_images}")

# === Load Label Paths ===
# Count total number of label files (e.g., .png format for segmentation masks)
all_label_paths = sorted(glob(os.path.join(label_path, "*.png")))
total_labels = len(all_label_paths)
print(f"Total Number of Labels: {total_labels}")

# === Match Images and Labels ===
# Assuming both are in matching order and of equal length
train_image_paths = all_image_paths[:total_images]
train_label_paths = all_label_paths[:total_labels]

# Preview label paths (for verification)
print("Sample label paths:")
for path in train_label_paths[:5]:
    print(path)

Improt Validation data

In [ ]:
# === CONFIGURATION ===
# Set the path to your validation images and labels
val_image_path = "/path/to/valid/images"   # <-- Replace with actual validation image folder
val_label_path = "/path/to/valid/labels"   # <-- Replace with actual validation label folder

# === Load Validation Image Paths ===
# Collect and sort all .jpg image files in the validation folder
val_all_image_paths = sorted(glob(os.path.join(val_image_path, "*.jpg")))
val_total_images = len(val_all_image_paths)
print(f"Total Number of Validation Images: {val_total_images}")

# === Load Validation Label Paths ===
# Collect and sort all .png label files in the validation folder
val_all_label_paths = sorted(glob(os.path.join(val_label_path, "*.png")))
val_total_labels = len(val_all_label_paths)
print(f"Total Number of Validation Labels: {val_total_labels}")

# === Match Images and Labels (by order) ===
# This assumes one-to-one correspondence between image and label files
Val1_image_paths = val_all_image_paths[:val_total_images]
Val1_label_paths = val_all_label_paths[:val_total_labels]

# Preview a few label paths to confirm loading
print("Sample validation label paths:")
for path in Val1_label_paths[:5]:
    print(path)

Reading ground_truth_masks for training and Validation

In [ ]:
# Please dont run this line if you would like to use the original size of input images.
desired_size=(640, 640)

In [ ]:
# === Load and Process Ground Truth Masks ===
# This dictionary will store binary masks where pixel > 0 is treated as True
ground_truth_masks = {}

for idx in range(len(train_label_paths)):
    # Read the label mask in grayscale
    gt_grayscale = cv2.imread(train_label_paths[idx], cv2.IMREAD_GRAYSCALE)

    # Resize the mask if desired_size is specified
    if desired_size is not None:
        gt_grayscale = cv2.resize(gt_grayscale, desired_size, interpolation=cv2.INTER_LINEAR)

    # Convert to binary mask (True where pixel > 0)
    ground_truth_masks[idx] = (gt_grayscale > 0)

# Optional: Print number of masks and preview a sample
print(f"Total ground truth masks loaded: {len(ground_truth_masks)}")
print("Example binary mask shape:", ground_truth_masks[0].shape)

In [ ]:
# === Load and Process Validation Ground Truth Masks ===
# This dictionary will store binary masks for validation data
ground_truth_masksv = {}

for idx in range(len(Val1_label_paths)):
    # Read the validation label mask in grayscale
    gt_grayscale = cv2.imread(Val1_label_paths[idx], cv2.IMREAD_GRAYSCALE)

    # Resize the mask if a desired size is specified
    if desired_size is not None:
        gt_grayscale = cv2.resize(gt_grayscale, desired_size, interpolation=cv2.INTER_LINEAR)

    # Convert to binary mask: True where pixel > 0
    ground_truth_masksv[idx] = (gt_grayscale > 0)

# Print summary
print(f"Total validation ground truth masks loaded: {len(ground_truth_masksv)}")
print("Example validation mask shape:", ground_truth_masksv[0].shape)

Import SAM model

In [ ]:
model_type = "vit_t"
checkpoint = "mobile_sam.pt"
device = 'cuda:0'

In [ ]:
from mobile_sam import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
import torch

# === Load Pretrained SAM Model ===
sam_model = sam_model_registry[model_type](checkpoint=checkpoint)

# === Device Setup ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sam_model.to(device)

# === Set to Training Mode/Inferencing Mode (for fine-tuning) ===
sam_model.eval()
sam_model.train()


#Step 1: Preprocess the images for training

In [ ]:
from collections import defaultdict
from segment_anything.utils.transforms import ResizeLongestSide

# === Initialize Resize Transformation ===
# This ensures images and masks are resized while preserving aspect ratio
transform = ResizeLongestSide(sam_model.image_encoder.img_size)

# === Containers for Processed Data ===
transformed_data = defaultdict(dict)  # holds resized + preprocessed inputs for SAM
ground_truth_masks = {}               # holds binary masks for training

# === Preprocess All Training Samples ===
for k in range(len(train_image_paths)):
    # Load RGB image and grayscale ground truth mask
    image_bgr = cv2.imread(train_image_paths[k])
    mask_gray = cv2.imread(train_lable_paths[k], cv2.IMREAD_GRAYSCALE)
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    # Apply resizing transformation to both image and mask
    resized_image = transform.apply_image(image_rgb)
    resized_mask = transform.apply_image(mask_gray)

    # Convert image to torch tensor and format as [1, 3, H, W]
    image_tensor = torch.as_tensor(resized_image, device=device).permute(2, 0, 1).unsqueeze(0).contiguous()

    # Preprocess for SAM (normalization, padding, etc.)
    input_tensor = sam_model.preprocess(image_tensor)

    # Store processed image and metadata
    transformed_data[k]['image'] = input_tensor
    transformed_data[k]['input_size'] = resized_image.shape[:2][::-1]  # (W, H)
    transformed_data[k]['original_image_size'] = image_rgb.shape[:2]   # (H, W)

    # Store resized mask as binary (bool)
    ground_truth_masks[k] = (resized_mask > 0)

print(f"Preprocessed {len(transformed_data)} training samples.")

In [ ]:
# === Initialize Resize Transformation ===
transform = ResizeLongestSide(sam_model.image_encoder.img_size)

# === Storage for Results ===
ground_truth_masksv = {}   # Ground truth validation masks (resized and binarized)
images_tuned_list = {}     # Resized RGB images used for prediction
masks_tuned_list = {}      # Predicted masks from SAM

# === Loop Through Validation Images ===
for s in range(len(Val1_image_paths)):
    # Load RGB image and grayscale ground truth mask
    image_bgr = cv2.imread(Val1_image_paths[s])
    gt_mask = cv2.imread(Val1_lable_paths[s], cv2.IMREAD_GRAYSCALE)
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    # Resize image and ground truth using SAM’s aspect-preserving transform
    resized_image = transform.apply_image(image_rgb)
    resized_mask = transform.apply_image(gt_mask)

    # Set resized image for SAM prediction
    predictor_tuned.set_image(resized_image)

    # Predict masks (without prompt)
    masks_tuned, _, _ = predictor_tuned.predict(
        point_coords=None,
        box=None,
        multimask_output=False
    )

    # Convert SAM prediction to binary mask
    pred_mask = masks_tuned[0]
    binary_mask = (pred_mask > 0).astype(np.float32)

    # Store outputs
    images_tuned_list[s] = resized_image
    masks_tuned_list[s] = binary_mask
    ground_truth_masksv[s] = (resized_mask > 0).astype(np.float32)

print(f"Predictions complete for {len(Val1_image_paths)} validation images.")


# Set up the optimizer, and Loss


In [ ]:
# === Training Configuration ===
keys = list(ground_truth_masks.keys())  # List of training sample indices
batch_size = 32                         # Adjust based on available GPU memory
num_epochs = 5                          # Total number of training epochs

# === Parameters to Fine-Tune ===
# You can choose to fine-tune all components or a subset
params_to_optimize = (
    list(sam_model.mask_decoder.parameters()) +        # Mask decoder (usually the main fine-tuning target)
    list(sam_model.image_encoder.parameters()) +       # Optional: include image encoder
    list(sam_model.prompt_encoder.parameters())        # Optional: include prompt encoder
)

# === Optimizer ===
# Adam optimizer with low learning rate for stable fine-tuning
optimizer = torch.optim.Adam(params_to_optimize, lr=1e-4, weight_decay=0)

# === Loss Function ===
# Binary Cross-Entropy with logits is used for binary segmentation tasks
loss_fn = torch.nn.BCEWithLogitsLoss()

# Fine tuning SAM by Training data

In [ ]:
from torch.utils.data import DataLoader

# === Validation DataLoader Setup ===
# Here we're using a list of file paths as the dataset, which will later need to be wrapped in a proper Dataset class
val_loader = DataLoader(Val1_image_paths, batch_size=batch_size, shuffle=False)

# === Basic Validation Dataset Checks ===

# Total number of validation examples
num_val_examples = len(Val1_image_paths)
print(f"Number of validation examples: {num_val_examples}")

# Number of items returned by val_loader.dataset (same as above since it's a list)
print(f"Number of examples in validation dataset (via DataLoader): {len(val_loader.dataset)}")

# Number of batches in the validation DataLoader
print(f"Number of batches in validation loader: {len(val_loader)}")

# === Safety Check ===
# Prevent training from continuing if validation data is empty
if num_val_examples == 0:
    raise ValueError("The validation dataset is empty. Please check your data paths.")

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# === Accuracy Calculation Function ===
def calculate_accuracy(predictions, targets):
    """
    Computes binary accuracy between predicted and ground truth masks.
    """
    binary_predictions = (predictions > 0.5).float()
    accuracy = (binary_predictions == targets).float().mean()
    return accuracy.item()

# === Batch Training Function ===
def train_on_batch(keys, batch_start, batch_end):
    """
    Trains the SAM mask decoder on a batch of images and masks.
    Returns batch loss and accuracy.
    """
    batch_losses = []
    batch_accuracies = []

    for k in keys[batch_start:batch_end]:
        # === Get input data and metadata
        input_image = transformed_data[k]['image'].to(device)
        input_size = transformed_data[k]['input_size']
        original_image_size = transformed_data[k]['original_image_size']

        # === Forward Pass (frozen encoders)
        with torch.no_grad():
            image_embedding = sam_model.image_encoder(input_image)
            sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
                points=None, boxes=None, masks=None
            )

        low_res_masks, _ = sam_model.mask_decoder(
            image_embeddings=image_embedding,
            image_pe=sam_model.prompt_encoder.get_dense_pe(),
            sparse_prompt_embeddings=sparse_embeddings,
            dense_prompt_embeddings=dense_embeddings,
            multimask_output=False
        )

        # === Resize prediction to original size
        upscaled_masks = sam_model.postprocess_masks(
            low_res_masks, input_size, original_image_size
        ).to(device)

        # === Resize ground truth mask to match output
        gt_np = ground_truth_masks[k].astype(np.uint8)
        resized_gt = cv2.resize(gt_np, upscaled_masks.shape[-2:][::-1], interpolation=cv2.INTER_NEAREST)
        gt_binary_mask = torch.tensor(resized_gt, dtype=torch.float32, device=device).unsqueeze(0).unsqueeze(0)

        # === Compute loss and update weights
        loss = loss_fn(upscaled_masks, gt_binary_mask)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # === Record metrics
        batch_losses.append(loss.item())
        batch_accuracies.append(calculate_accuracy(torch.sigmoid(upscaled_masks), gt_binary_mask))

    return batch_losses, batch_accuracies

# === Epoch Training Loop ===
losses, accuracies = [], []

for epoch in range(num_epochs):
    epoch_losses = []
    epoch_accuracies = []

    print(f"\n--- EPOCH {epoch + 1}/{num_epochs} ---")

    for batch_start in range(0, len(keys), batch_size):
        batch_end = min(batch_start + batch_size, len(keys))
        batch_losses, batch_accuracies = train_on_batch(keys, batch_start, batch_end)

        batch_loss = mean(batch_losses)
        batch_accuracy = mean(batch_accuracies)
        epoch_losses.append(batch_loss)
        epoch_accuracies.extend(batch_accuracies)

        print(f'Batch [{batch_start + 1}–{batch_end}] | Loss: {batch_loss:.6f} | Accuracy: {batch_accuracy:.4f}')

    # === End of Epoch ===
    mean_train_loss = mean(epoch_losses)
    mean_train_accuracy = mean(epoch_accuracies)
    losses.append(mean_train_loss)
    accuracies.append(mean_train_accuracy)

    print(f'\nEpoch {epoch + 1} Summary:')
    print(f'➤ Mean Training Loss: {mean_train_loss:.6f}')
    print(f'➤ Mean Training Accuracy: {mean_train_accuracy:.4f}')

    # Clear cache to manage memory
    torch.cuda.empty_cache()


In [ ]:
print(type(ground_truth_masksv))
print(ground_truth_masksv.keys() if isinstance(ground_truth_masksv, dict) else len(ground_truth_masksv))


#Step 3: Testing fine-tuned SAM model

In [ ]:
# Set the paths to your test images and labels
test_image_dir = "/path/to/test/images"   # <-- Update this path
test_label_dir = "/path/to/test/labels"   # <-- Update this path

# === Load and Sort Test Image Paths ===
# Collect all test image files (e.g., .jpg)
all_test_image_paths = sorted(glob(os.path.join(test_image_dir, "*.jpg")))
test_total_images = len(all_test_image_paths)
print(f"Total Number of Test Images: {test_total_images}")

# === Load and Sort Test Label Paths ===
# Collect all test label files (e.g., .png masks)
all_test_label_paths = sorted(glob(os.path.join(test_label_dir, "*.png")))
test_total_labels = len(all_test_label_paths)
print(f"Total Number of Test Labels: {test_total_labels}")# === Count test samples ===
test_total_images = len(Test_image_paths)
test_total_labels = len(Test_lable_paths)

# === Dictionary to store binary test masks ===
ground_truth_test_masks = {}

# === Load and resize ground truth masks ===
for k in range(test_total_labels):
    # Read mask as single-channel grayscale
    gt_gray = cv2.imread(Test_lable_paths[k], cv2.IMREAD_GRAYSCALE)

    # Resize if desired size is specified
    if desired_size is not None:
        gt_gray = cv2.resize(gt_gray, desired_size, interpolation=cv2.INTER_NEAREST)

    # Convert to binary mask (float32 format)
    ground_truth_test_masks[k] = (gt_gray > 0).astype(np.float32)

print(f" Loaded {len(ground_truth_test_masks)} ground truth test masks.")

# === Match Image and Label Paths ===
# These lists can now be used for DataLoader or evaluation
Test_image_paths = all_test_image_paths[:test_total_images]
Test_label_paths = all_test_label_paths[:test_total_labels]

# Optional: Print a few samples to verify
print("Sample test image path:", Test_image_paths[0] if Test_image_paths else "No images found")
print("Sample test label path:", Test_label_paths[0] if Test_label_paths else "No labels found")


Box for Test *data*

Ground_truth_test_masks

In [ ]:
# Dictionary to hold ground truth binary masks for test data
ground_truth_test_masks = {}

# === Load and Process Each Test Mask ===
for idx in range(len(Test_label_paths)):
    # Read label image in color (3-channel); expected mask is in the red channel
    gt_color = cv2.imread(Test_label_paths[idx])

    # Extract the red channel only and convert to binary mask
    # Note: OpenCV loads in BGR, so red is at index 2
    binary_mask = (gt_color[:, :, 2] > 0).astype(np.float32)

    # Resize if specified
    if desired_size is not None:
        binary_mask = cv2.resize(binary_mask, desired_size, interpolation=cv2.INTER_NEAREST)

    # Store in dictionary
    ground_truth_test_masks[idx] = binary_mask

print(f"Loaded {len(ground_truth_test_masks)} ground truth test masks.")

Load the trained model with best parameters

Prediction using Fine_tuned model

In [ ]:
# === Output containers ===
images_tuned_list = {}  # Stores resized RGB input images
masks_tuned_list = {}   # Stores binary segmentation masks

# === Run inference on all test images ===
for k in range(test_total_images):
    # Load image from disk
    image_bgr = cv2.imread(Test_image_paths[k])  # use Test_image_paths here
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    # Optional resizing
    if desired_size is not None:
        image_rgb = cv2.resize(image_rgb, desired_size, interpolation=cv2.INTER_LINEAR)

    # Set image for SAM predictor
    predictor_tuned.set_image(image_rgb)

    # Run prediction (no prompt input)
    masks_tuned, _, _ = predictor_tuned.predict(
        point_coords=None,
        box=None,
        multimask_output=False  # Only return most confident mask
    )

    # Convert first predicted mask to binary
    pred_mask = masks_tuned[0]  # shape: (H, W)
    binary_mask = (pred_mask > 0).astype(np.float32)

    # Store results
    images_tuned_list[k] = image_rgb
    masks_tuned_list[k] = binary_mask

print(f"Inference complete on {test_total_images} test images.")

Plot results on all of the Test data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# === Configuration ===
n_images = len(images_tuned_list)
n_cols = 4  # Images per row
n_rows = (n_images + n_cols - 1) // n_cols  # Automatically calculate needed rows

# === Create subplot grid ===
fig, axs = plt.subplots(n_rows, n_cols, figsize=(4 * n_cols, 4 * n_rows))

# Ensure axs is always 2D (e.g., when n_rows = 1)
axs = np.atleast_2d(axs)

# === Loop through and display each image + overlay ===
for i in range(n_rows):
    for j in range(n_cols):
        index = i * n_cols + j
        ax = axs[i, j]

        if index < n_images:
            # Load image and binary mask
            image = images_tuned_list[index]
            mask = masks_tuned_list[index]

            # Display the RGB image
            ax.imshow(image, interpolation='none')

            # Create blue overlay for mask
            blue_overlay = np.stack([
                np.zeros_like(mask),
                np.zeros_like(mask),
                (mask > 0).astype(np.float32)
            ], axis=-1)

            # Overlay with transparency
            ax.imshow(blue_overlay, alpha=0.5)

        # Turn off axes for visual cleanliness
        ax.axis('off')

# === Adjust layout ===
plt.subplots_adjust(wspace=0.03, hspace=0.03)
plt.tight_layout()
plt.show()

In [ ]:
import torch
import numpy as np
from sklearn.metrics import auc, roc_curve

# === Binary Metrics for One Prediction ===
def binary_segmentation_metrics(predictions, targets):
    """
    Computes binary segmentation metrics for a single predicted mask vs ground truth.
    Inputs:
        predictions (numpy array): predicted mask, float32, range [0,1] or binary
        targets (numpy array): ground truth mask, binary (0 or 1)
    Returns:
        Tuple of metrics: accuracy, precision, recall, F1-score, IoU, kappa, FP, FN, TP, TN, dice
    """
    # Flatten and convert to binary
    predictions = predictions.squeeze()
    targets = targets.squeeze()

    predictions_binary = (predictions > 0.5).astype(int)
    targets_binary = targets.astype(int)

    # Confusion matrix components
    TP = np.sum((predictions_binary == 1) & (targets_binary == 1))
    FP = np.sum((predictions_binary == 1) & (targets_binary == 0))
    FN = np.sum((predictions_binary == 0) & (targets_binary == 1))
    TN = np.sum((predictions_binary == 0) & (targets_binary == 0))

    # Metrics with small epsilon to avoid division by zero
    eps = 1e-5
    accuracy = (TP + TN + eps) / (TP + FP + FN + TN + eps)
    precision = (TP + eps) / (TP + FP + eps)
    recall = (TP + eps) / (TP + FN + eps)
    f_score = 2 * (precision * recall) / (precision + recall + eps)
    dice = (2 * TP + eps) / (2 * TP + FP + FN + eps)
    iou = (TP + eps) / (TP + FP + FN + eps)

    # Cohen’s kappa
    total = TP + FP + FN + TN
    p_o = (TP + TN) / total
    p_e = ((TP + FP) * (TP + FN) + (FN + TN) * (FP + TN)) / (total ** 2)
    kappa = (p_o - p_e) / (1 - p_e + eps)

    return accuracy, precision, recall, f_score, iou, kappa, FP, FN, TP, TN, dice

# === Average Metrics Across Dataset ===
def calculate_average_metrics(predictions_list, targets_list):
    """
    Computes average binary segmentation metrics across a dataset.
    Inputs:
        predictions_list: dictionary or list of predicted masks
        targets_list: dictionary or list of ground truth masks
    Returns:
        Dictionary of averaged metrics
    """
    num_masks = len(predictions_list)

    total_metrics = {
        'accuracy': 0.0, 'precision': 0.0, 'recall': 0.0, 'f_score': 0.0,
        'iou': 0.0, 'kappa': 0.0, 'FP': 0, 'FN': 0, 'MAR': 0.0, 'FAR': 0.0, 'dice': 0.0
    }

    for i in range(num_masks):
        pred = predictions_list[i]
        gt = targets_list[i]
        metrics = binary_segmentation_metrics(pred, gt)

        # Accumulate each metric
        for metric_name, value in zip(total_metrics.keys(), metrics):
            total_metrics[metric_name] += value

        # Add False Negative Rate (Missed Alarm Rate, MAR) and False Alarm Rate (FAR)
        TP, TN, FP, FN = metrics[8], metrics[9], metrics[6], metrics[7]
        total_metrics['MAR'] += FN / (FN + TP + 1e-5)
        total_metrics['FAR'] += FP / (FP + TN + 1e-5)

    # Compute mean for each metric
    avg_metrics = {k: v / num_masks for k, v in total_metrics.items()}

    return avg_metrics

# === Example Usage ===
# Evaluate the SAM predictions vs. ground truth test masks
avg_metrics = calculate_average_metrics(masks_tuned_list, ground_truth_test_masks)

# Print results
print("\n=== Average Metrics on Test Set ===")
for metric_name, value in avg_metrics.items():
    print(f"{metric_name.upper():<8}: {value:.4f}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from skimage.color import rgb2gray
import math

# === Configuration ===
border_color = 'black'
border_width = 0.35
alpha_gray = 0.5  # transparency for base grayscale image
alpha_tp = 0.5    # transparency for True Positives (blue)
alpha_fa = 0.6    # False Alarms (green)
alpha_ma = 0.6    # Missed Alarms (magenta)

# === Determine Grid Layout ===
num_images = len(images_tuned_list)
cols = 4
rows = math.ceil(num_images / cols)

# === Setup Plot ===
fig, axs = plt.subplots(rows, cols, figsize=(4 * cols, 4 * rows))
axs = axs.flatten()  # flatten 2D axes array for 1D indexing

# === Choose Which Images to Display (all by default) ===
selected_indices = list(range(num_images))

# === Visualization Loop ===
for i, index in enumerate(selected_indices):
    if index >= num_images:
        break

    # Convert image to grayscale
    gray_image = rgb2gray(images_tuned_list[index])

    # Get predicted and ground truth masks
    pred_mask = masks_tuned_list[index]
    gt_mask = ground_truth_test_masks[index]

    ax = axs[i]
    ax.imshow(gray_image, cmap='gray', interpolation='none', alpha=alpha_gray)

    # === Mask Overlays ===

    # True Positives: prediction and ground truth both 1 (blue)
    tp_mask = (pred_mask == 1) & (gt_mask == 1)
    tp_rgba = np.stack([np.zeros_like(tp_mask), np.zeros_like(tp_mask), tp_mask], axis=-1)
    tp_rgba = np.concatenate([tp_rgba, tp_mask[..., None].astype(float)], axis=-1)
    ax.imshow(tp_rgba, alpha=alpha_tp)

    # False Alarms: prediction is 1, ground truth is 0 (green)
    fa_mask = (pred_mask == 1) & (gt_mask == 0)
    fa_rgba = np.stack([np.zeros_like(fa_mask), fa_mask, np.zeros_like(fa_mask)], axis=-1)
    fa_rgba = np.concatenate([fa_rgba, fa_mask[..., None].astype(float)], axis=-1)
    ax.imshow(fa_rgba, alpha=alpha_fa)

    # Missed Alarms: prediction is 0, ground truth is 1 (magenta)
    ma_mask = (pred_mask == 0) & (gt_mask == 1)
    ma_rgba = np.stack([ma_mask, np.zeros_like(ma_mask), ma_mask], axis=-1)
    ma_rgba = np.concatenate([ma_rgba, ma_mask[..., None].astype(float)], axis=-1)
    ax.imshow(ma_rgba, alpha=alpha_ma)

    # Add black border
    ax.add_patch(Rectangle((0, 0), gray_image.shape[1], gray_image.shape[0],
                           linewidth=border_width, edgecolor=border_color, facecolor='none'))

    ax.axis('off')

# === Turn Off Any Unused Axes ===
for j in range(len(selected_indices), len(axs)):
    axs[j].axis('off')

# === Save and Show ===
plt.subplots_adjust(wspace=0.02, hspace=0.02)
output_path = "./ViTSAM_Evaluation_Overlay.png"  # <-- Update path as needed
plt.savefig(output_path, dpi=600, bbox_inches='tight')
plt.show()

print(f"Visualization saved to: {output_path}")


Validation

In [ ]:
import numpy as np

# === Metric Computation Function ===
def binary_segmentation_metrics(predictions, targets):
    predictions = predictions.squeeze()
    targets = targets.squeeze()
    predictions_binary = (predictions > 0.5).astype(int)
    targets_binary = targets.astype(int)

    TP = np.sum((predictions_binary == 1) & (targets_binary == 1))
    FP = np.sum((predictions_binary == 1) & (targets_binary == 0))
    FN = np.sum((predictions_binary == 0) & (targets_binary == 1))
    TN = np.sum((predictions_binary == 0) & (targets_binary == 0))

    eps = 1e-5
    accuracy = (TP + TN + eps) / (TP + FP + FN + TN + eps)
    precision = (TP + eps) / (TP + FP + eps)
    recall = (TP + eps) / (TP + FN + eps)
    f_score = 2 * (precision * recall) / (precision + recall + eps)
    dice = (2 * TP + eps) / (2 * TP + FP + FN + eps)
    iou = (TP + eps) / (TP + FP + FN + eps)

    total = TP + FP + FN + TN
    p_o = (TP + TN) / total
    p_e = ((TP + FP) * (TP + FN) + (FN + TN) * (FP + TN)) / (total ** 2)
    kappa = (p_o - p_e) / (1 - p_e + eps)

    return accuracy, precision, recall, f_score, iou, kappa, FP, FN, TP, TN, dice

# === Categorization Function ===
def categorize_metric(value, metric):
    categories = {
        'IoU': [(0.90, 'Excellent'), (0.85, 'Good'), (0.75, 'Fair'), (0.65, 'Poor'), (0, 'Unacceptable')],
        'Precision': [(0.95, 'Excellent'), (0.85, 'Good'), (0.65, 'Moderate'), (0, 'Fail')],
        'Kappa': [(0.88, 'Excellent'), (0.78, 'Good'), (0.68, 'Moderate'), (0, 'Fail')],
        'F-Score': [(0.88, 'Excellent'), (0.78, 'Good'), (0.68, 'Moderate'), (0, 'Fail')],
        'Recall': [(0.88, 'Excellent'), (0.78, 'Good'), (0.68, 'Moderate'), (0, 'Fail')]
    }

    if metric in categories:
        for threshold, label in categories[metric]:
            if value >= threshold:
                return label
    return "Unknown"

# === Compute Metrics for All Images ===
def compute_all_metrics(predictions_list, targets_list):
    all_metrics = []
    for i in range(len(predictions_list)):
        metrics = binary_segmentation_metrics(predictions_list[i], targets_list[i])
        metric_names = ['Accuracy', 'Precision', 'Recall', 'F-Score', 'IoU', 'Kappa', 'FP', 'FN', 'TP', 'TN', 'Dice']
        metric_dict = dict(zip(metric_names, metrics))

        # Add categorized versions
        for name in ['IoU', 'Precision', 'Kappa', 'F-Score', 'Recall']:
            metric_dict[f'{name}_Category'] = categorize_metric(metric_dict[name], name)

        all_metrics.append(metric_dict)
    return all_metrics

# === Summarize Counts and Percentages ===
def summarize_category_counts(metrics_list, metric_name, labels):
    counts = {label: 0 for label in labels}
    total = len(metrics_list)

    for m in metrics_list:
        category = m.get(f"{metric_name}_Category")
        if category in counts:
            counts[category] += 1

    # Print summary
    print(f"Counts and Percentages of Images in Each Category for {metric_name}:")
    for label in labels:
        count = counts[label]
        percentage = (count / total) * 100
        print(f"{label}: {count} ({percentage:.2f}%)")
    print()

# === Run All ===
metrics_list = compute_all_metrics(masks_tuned_list, ground_truth_test_masks)

summarize_category_counts(metrics_list, 'IoU',        ['Excellent', 'Good', 'Fair', 'Poor', 'Unacceptable'])
summarize_category_counts(metrics_list, 'Precision',  ['Excellent', 'Good', 'Moderate', 'Fail'])
summarize_category_counts(metrics_list, 'Kappa',      ['Excellent', 'Good', 'Moderate', 'Fail'])
summarize_category_counts(metrics_list, 'F-Score',    ['Excellent', 'Good', 'Moderate', 'Fail'])
summarize_category_counts(metrics_list, 'Recall',     ['Excellent', 'Good', 'Moderate', 'Fail'])